# Truth or Dare Game dengan Face Detection

<p>Program permainan Truth or Dare interaktif menggunakan python sederhana dengan fitur deteksi gerakan kepala menggunakan MediaPipe dan OpenCV.</p>

Fitur:
<ul>
<li>Deteksi wajah real-time</li>
<li>Kontrol permainan dengan gerakan kepala</li>
<li>Sistem pertanyaan Truth dan tantangan Dare</li>
</ul>

<p>Cara Penggunaan.</p>

Gerakan kepala:

<li>Kiri: Memilih Dare</li>
<li>Kanan: Memilih Truth</li>
<li>Tengah: Posisi netral</li><br />

Kontrol keyboard:

<li>'T': Truth</li>
<li>'D': Dare</li>
<li>'R': Random</li>
<li>'ESC': Keluar</li>

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.6 MB 342.2 kB/s eta 0:00:30
   - -------------------------------------- 0.5/10.6 MB 342.2 kB/s eta 0:00:30
   - -------------------------------------- 0.5/10.6 MB 342.2 kB/s eta 0:00:30
   -- ------------------------------------- 0.8/10.6 MB 335.4 kB/s eta 0:00:30
   -- ------------------------------------- 0.8/10.6 MB 335.4 kB/s eta 0:00:30
   -- ------------------------------------- 0

# Setup VideoCapture library cv2

In [317]:
import cv2

index = 0
while True:
    cap = cv2.VideoCapture(index)
    if not cap.isOpened():
        print(f"Kamera dengan index {index} tidak tersedia.")
        break
    print(f"Kamera dengan index {index} tersedia.")
    index += 1
    cap.release()

# Kamera 0 artinya kamera internal
# Kamera 1 artinya kamera eksternal
# Kamera 2 artinya kamera tidak tersedia


Kamera dengan index 0 tersedia.
Kamera dengan index 1 tersedia.
Kamera dengan index 2 tidak tersedia.


# Library

<p>Menggunakan Library random, openCv, mediapipe, os, numpy, dan pygame</p>

In [320]:
import random
import cv2
import mediapipe as mp
import numpy as np
import pygame
import time

# Pertanyaan dan konfigurasi kamera, sensitivitas dan lainnya

In [322]:
# Konfigurasi
CONFIG = {
    'camera_width': 800,
    'camera_height': 600,
    'detection_confidence': 0.5,
    'head_sensitivity': 0.15,
    'countdown_duration': 10,
    'overlay_opacity': 0.3
}

# Daftar pertanyaan Truth dan tantangan Dare yang lebih variatif
truth_questions = [
    "Apa pengalaman paling memalukan yang pernah kau alami?",
    "Siapa orang yang paling kau kagumi dan kenapa?",
    "Pernahkah ente berbohong kepada kawanmu? Ceritakan singkat!",
    "Apa impian terbesar yang ingin antum wujudkan?",
    "Kapan terakhir kali kau nangis?",
    "Apa hal gila yang pernah kau lakuin?",
    "Siapa orang yang berpengaruh dalam hidup kau?"
]

dare_challenges = [
    "Senyum pepsoden selama 7 detik!",
    "Ekspresi jelek dulu boss!",
    "Spill mantan terrr-mu!",
    "Coba bicara dengan aksen asing selama beberapa kalimat!.",
    "Joget pisang dulu 5 detik!",
    "Kedipkan matamu 6 kali!",
    "Lakukan gerakan tarian TikTok paling viral !",
]


# Setup audio, Setup Backsound, Fungsi Random, dll

In [324]:
# Inisialisasi MediaPipe dan Pygame
mp_face_detection = mp.solutions.face_detection

def setup_audio():
    """Inisialisasi dan konfigurasi audio"""
    try:
        pygame.mixer.init()
        pygame.mixer.music.set_volume(0.5)
    except Exception as e:
        print(f"Error inisialisasi audio: {e}")

def play_sound(choice):
    """Putar efek suara berdasarkan pilihan"""
    sound_map = {
        "Truth": "Audio/truth_sound.mp3",
        "Dare": "Audio/dare_sound.mp3"
    }
    try:
        sound_path = sound_map.get(choice, "Audio/truth_sound.mp3")
        if os.path.exists(sound_path):
            pygame.mixer.music.load(sound_path)
            pygame.mixer.music.play()
        else:
            print(f"File suara tidak ditemukan: {sound_path}")
    except Exception as e:
        print(f"Error memainkan suara: {e}")

def get_truth_or_dare():
    """Pilih acak antara Truth atau Dare"""
    return random.choice(['Truth', 'Dare'])

def get_random_item(choice):
    """Ambil pertanyaan atau tantangan secara acak"""
    return random.choice(truth_questions if choice == "Truth" else dare_challenges)

def wrap_text(frame, text, position, color, font_scale=1, line_height=50, thickness=2):
    """Fungsi untuk membuat teks yang dapat dibungkus"""
    words = text.split()
    x, y = position
    line = ""
    for word in words:
        test_line = f"{line} {word}".strip()
        size = cv2.getTextSize(test_line, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[0]
        
        if size[0] > frame.shape[1] - x:
            cv2.putText(frame, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 
                        font_scale, color, thickness, cv2.LINE_AA)
            y += line_height
            line = word
        else:
            line = test_line
    
    cv2.putText(frame, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 
                font_scale, color, thickness, cv2.LINE_AA)


# Fungsi Head Direction Set, Trigger by Head, dan Overlay pada Frame

In [329]:
def detect_head_direction(frame, detection):
    """Deteksi arah kepala dengan presisi lebih tinggi"""
    bboxC = detection.location_data.relative_bounding_box
    ih, iw, _ = frame.shape
    x, y, w, h = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                  int(bboxC.width * iw), int(bboxC.height * ih))

    center_x = x + w // 2
    sensitivity_threshold = iw * CONFIG['head_sensitivity']
    
    if center_x < iw * 0.5 - sensitivity_threshold:
        return "Left"
    elif center_x > iw * 0.5 + sensitivity_threshold:
        return "Right"
    return "Neutral"

def trigger_action_by_head_direction(head_direction):
    """Tentukan aksi berdasarkan arah kepala"""
    if head_direction == "Left":
        choice = "Truth"
    elif head_direction == "Right":
        choice = "Dare"
    else:
        choice = get_truth_or_dare()
    
    result = get_random_item(choice)
    return choice, result


# Setting Gradient dan Warna

In [332]:
def apply_gradient_overlay(frame, color1, color2):
    """Membuat overlay dengan efek gradient"""
    gradient = np.zeros_like(frame, dtype=np.uint8)
    for i in range(frame.shape[0]):
        alpha = i / float(frame.shape[0])  # Membuat alpha dari 0 ke 1
        color = np.array(color1) * (1 - alpha) + np.array(color2) * alpha
        gradient[i, :, :] = color

    # Terapkan efek gradient ke frame
    frame = cv2.addWeighted(frame, 1 - CONFIG['overlay_opacity'], gradient, CONFIG['overlay_opacity'], 0)
    return frame

def apply_creative_overlay(frame, choice):
    """Tambahkan overlay kreatif dengan efek keburaman, gradient, dan animasi teks"""
    color1 = (0, 128, 255) if choice == "Truth" else (255, 0, 128)
    color2 = (255, 255, 255)  # Warna kedua untuk gradientnya

    # Terapkan gradient overlay
    frame = apply_gradient_overlay(frame, color1, color2)

    # Terapkan keburaman pada overlay (jika diperlukan)
    overlay = frame.copy()
    # overlay = cv2.GaussianBlur(overlay, (15, 15), 0)

    # Efek transparansi
    frame = cv2.addWeighted(overlay, CONFIG['overlay_opacity'], 
                            frame, 1 - CONFIG['overlay_opacity'], 0)

    return frame

# Fungsi Logik dan Filter Truth or Dare

In [340]:
def main_filter_logic():
    
    """Fungsi utama filter dan logika"""
    
    setup_audio()
    
    # Konfigurasi kamera
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CONFIG['camera_width'])
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CONFIG['camera_height'])

    if not cap.isOpened():
        print("Kamera tidak ditemukan!")
        return

    with mp.solutions.face_detection.FaceDetection(
        min_detection_confidence=CONFIG['detection_confidence']) as face_detection:
        
        choice, result = None, ""
        show_text = False
        last_action_time = 0
        countdown_active = False

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Gagal membaca frame!")
                break

            # Buat salinan frame untuk manipulasi
            display_frame = frame.copy()

            if not countdown_active:
                # Teks Judul Filter Truth or Dare
                cv2.putText(display_frame, f"Filter Truth Or Dare",(100, 120),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 10, cv2.LINE_AA)  # Outline hitam
                cv2.putText(display_frame, f"Filter Truth Or Dare",(100, 120),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 6, cv2.LINE_AA)  # Teks utama
                
                # Menampilkan teks petunjuk penggunaan
                cv2.putText(display_frame, 
                        "Press 'T': Truth, 'D': Dare, 'R': Random, 'ESC': Exit",
                        (10, display_frame.shape[0] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            

            # Deteksi wajah
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(frame_rgb)

            """
                Tampilkan pertanyaan ataupun tantangan di atas kepala dengan menggunakan koordinat bounding box yang telah di set teks tsb 
                agar selalu tracking dan berada diatas kepala.
            """
            if results.detections:
                for detection in results.detections:
                    head_direction = detect_head_direction(frame, detection)
                    
                    # Ambil koordinat bounding box wajah
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    x, y, w, h = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                                  int(bboxC.width * iw), int(bboxC.height * ih))
                    
                    # Hitung posisi teks agar selalu di atas kepala
                    text_x = x + w // 2 - 100  # Posisi horizontal (tengah wajah)
                    text_y = y - 20      # Posisi vertikal (sedikit di atas kepala)
                    
                    # Pastikan teks tidak keluar dari frame (atas layar)
                    text_y = max(30, text_y)
                    
                    # Tampilkan teks pertanyaan/tantangan di atas kepala
                    if countdown_active:
                        wrap_text(display_frame, result, (text_x - 100, text_y), 
                                  (255, 255, 255), font_scale=0.8, line_height=30)

            # Deteksi kepala
            if results.detections:
                for detection in results.detections:
                    head_direction = detect_head_direction(frame, detection)
                    
                    current_time = time.time()
                    if (head_direction in ["Left", "Right"] and 
                        current_time - last_action_time > 2 and 
                        not countdown_active):
                        
                        # Pilih dan tentukan pertanyaan/tantangan
                        choice, result = trigger_action_by_head_direction(head_direction)
                        last_action_time = current_time
                        show_text = True
                        
                        play_sound(choice)
                        countdown_active = True

            # Kontrol Trigger dengan keyboard
            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC
                break
            elif key in [ord('t'), ord('d'), ord('r')] and not countdown_active:
                if key == ord('t'):
                    choice = "Truth"
                elif key == ord('d'):
                    choice = "Dare"
                else:
                    choice = get_truth_or_dare()  # Fungsi Random
            
                result = get_random_item(choice)  # Ambil teks tantangan
                play_sound(choice)  # Mainkan suara tantangan
            
                # Sinkronkan filter dan teks
                countdown_active = True
                last_action_time = time.time()
                # print(f"Keyboard Trigger: {choice}")  # Debug log

            # Proses tampilan saat countdown aktif
            if countdown_active:
                if results.detections is not None:  # Deteksi tidak None
                    for detection in results.detections:
                        # Ambil posisi kepala seperti sebelumnya
                        bboxC = detection.location_data.relative_bounding_box
                        ih, iw, _ = frame.shape
                        x, y, w, h = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                                      int(bboxC.width * iw), int(bboxC.height * ih))
                
                        # Set juga posisi teks koordinat berikut ditas kepala mengikuti settingan bounding box sebelumnya
                        text_x = x + w // 2 - 100
                        text_y = y - 20
                        text_y = max(30, text_y)
                
                        # Tampilkan teks pertanyaan/tantangan di atas kepala
                        if countdown_active:
                            wrap_text(display_frame, result, (text_x - 100, text_y),
                                      (255, 255, 255), font_scale=0.8, line_height=30)

                
                # # Overlay filter
                # color = (0, 128, 255) if choice == "Truth" else (255, 0, 128)
                # overlay = display_frame.copy()
                # cv2.rectangle(overlay, (0, 0), (display_frame.shape[1], display_frame.shape[0]), color, -1)
                # display_frame = cv2.addWeighted(overlay, 0.8, display_frame, 0.2, 0)

                # Menerapkan custom overlay kreatif pada frame berdasarkan pilihan
                display_frame = apply_creative_overlay(display_frame, choice)
                
                # Tampilkan pilihan (Truth/Dare)
                cv2.putText(display_frame, choice.upper(), (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 8)
                
                # Tampilkan pertanyaan/tantangan
                # wrap_text(display_frame, result, (50, 200), 
                #           (255, 255, 255), font_scale=1, line_height=50)
                
                # Proses countdown
                remaining_time = CONFIG['countdown_duration'] - int(time.time() - last_action_time)
                
                # Tampilkan teks dan countdown
                cv2.putText(display_frame, f"Next: {remaining_time}",(50, 450),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 4, cv2.LINE_AA)  # Teks utama
                
                # Reset if countdown selesai
                if remaining_time <= 0:
                    countdown_active = False
                    show_text = False

            # Tampilkan frame
            cv2.imshow("Truth or Dare Filter", display_frame)

    cap.release()
    cv2.destroyAllWindows()
    pygame.mixer.quit()

# Jalankan program
if __name__ == "__main__":
    main_filter_logic()